In [4]:
import numpy as np
from scipy.stats import norm
import pandas as pd
import datetime
import os

$\textbf{PDE:}$

$$\frac{\partial V}{\partial t}+r\left( S \frac{\partial V}{\partial S}-V\right)+\frac{1}{2}\sigma^2S^2\frac{\partial^2 V}{\partial S^2}=0$$

$\textbf{SOLUTION:}$

<!-- $$ V(S, t) = e^{-r(T-t)}\left[S_0 \; \text{N}(\pm d_1) - K\; \text{N}(\pm d_2)\right]$$ -->

$$C(S, t) = S_0N(d_1) - Ke^{-r(T-t)}N(d_2)$$

$$P(S, t) = Ke^{-r(T-t)}N(-d_2) - S_0N(-d_1)$$

$\text{where}\; d_1 \; \& \; d_2 \; \text{are given by}$

$$ d_1 = \frac{\ln\left(\frac{S}{K}\right) + \left(r + \frac{\sigma^2}{2}\right)(T-t)}{\sigma\sqrt{T-t}} $$

$\text{and} $

$$d_2 = d_1 - \sigma\sqrt{T-t}$$

In [5]:
def european_option_price_BS(S0: float, r: float, sigma: float, T: float, K: float, opt_type: str) -> float:
    """
    Calculates the price of a European option (call or put) using the Black-Scholes formula.
    
    Parameters:
    - S0: float
        The initial stock price.
    - K: float
        The strike price of the option.
    - T: float
        The time to expiration in years.
    - r: float
        The risk-free interest rate.
    - sigma: float
        The volatility of the stock.
    - opt_type: str
        The type of option, either "call" or "put".
    
    Returns:
    - float
        The price of the European option.
    
    Raises:
    - ValueError
        If any of the input parameters are invalid or if the option type is not recognized.
    """
    
    # Validating the input parameters
    if S0 <= 0 or K <= 0 or T <= 0 or r < 0 or sigma <= 0:
        raise ValueError("Input parameters should be positive and non-zero.")
    if opt_type not in ["call", "put"]:
        raise ValueError("Option type must be either 'call' or 'put'.")

    # Calculating d1 and d2
    d1 = (np.log(S0 / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    
    # Calculating the option price based on the type
    if opt_type == "call":
        option_price = S0 * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    else:  # opt_type == "put"
        option_price = K * np.exp(-r * T) * norm.cdf(-d2) - S0 * norm.cdf(-d1)
    
    return option_price

In [6]:
# Example usage:    
S0 = 180
r = 0.02
sigma = 0.25
T = 1
K = 182


opt_type = "call"
option_price = european_option_price_BS(S0, r, sigma, T, K, opt_type)
print(f"The price of the American {opt_type.capitalize()} option is: {option_price}")

opt_type = "put"
option_price = european_option_price_BS(S0, r, sigma, T, K, opt_type)
print(f"The price of the American {opt_type.capitalize()} option is: {option_price}")

The price of the American Call option is: 18.639272426282943
The price of the American Put option is: 17.035430968112422


In [ ]:
for file in os.listdir('../option_data/spy_data'):
    if file[-4:] == '.csv':
        print(file)
        df = pd.read_csv('../option_data/spy_data/' + file)        
        
        # moving to datetime and making features
        df['quote_datetime'] = pd.to_datetime(df['quote_datetime'])
        df['expiration'] = pd.to_datetime(df['expiration'])
        df['quote_date'] = df['quote_datetime'][0].date()
        df['quote_date'] = pd.to_datetime(df['quote_date'])
        
        # getting only 4:00 quotes
        eod = datetime.datetime.combine(df['quote_datetime'][0].date(), datetime.time(16,0, 0))
        df = df.loc[df['quote_datetime'] == eod]
        
        # getting time to expiration and moneyness
        df['T'] = df['expiration'] - df['quote_date']
        df['T'] = df['T'].dt.days
        df['moneyness'] = df['active_underlying_price'] / df['strike'] 

        
        # filtering for research paper criteria
        df = df.loc[(df['close']!=0) & (df['implied_volatility']!=0) & (df['T']>=20) & (df['T']<=365) & (df['moneyness']>0.7) & (df['moneyness']<1.3)]

        calls = df.loc[df['option_type']=='C'][['T', 'moneyness', 'implied_volatility', 'active_underlying_price', 'strike']]
        puts = df.loc[df['option_type']=='P'][['T', 'moneyness', 'implied_volatility', 'active_underlying_price', 'strike']]